In [1]:
import sqlite3
import pandas as pd
import io

In [2]:
with open("database.sql", "r") as file:
    sql_script = file.read()

connection = sqlite3.connect(":memory:")

with connection:
    connection.executescript(sql_script)

In [3]:
query = """
            SELECT * 
            FROM companhia_aerea
        """

df = pd.read_sql_query(query, connection)
df

,id_companhia_aerea,nome_companhia,regra_tarifaria
0,1,LATAM AIRLINES BRASIL,None
1,2,GOL Linhas Aéreas Inteligentes,https://www.voegol.com.br/pt/informacoes/tarifas
2,3,AZUL LINHAS AÉREAS BRASILEIRA,https://www.voeazul.com.br/para-sua-viagem/inf...


## Consulta 1: Órgão que comprou o bilhete mais caro gerado pela LATAM

In [4]:
query = """
            SELECT DISTINCT codigo_orgao, nome_orgao, valor_bilhete
            FROM orgao
            NATURAL JOIN orgao_bilhete
            NATURAL JOIN bilhete
            NATURAL JOIN companhia_aerea
            WHERE nome_companhia == "LATAM AIRLINES BRASIL"
            ORDER BY
                valor_bilhete DESC
            LIMIT 1
            
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao,valor_bilhete
0,78055,Secretaria Especial de Articulação e Monitoram...,5847.14


## Consulta 2: Aviões que tem mais de 200 assentos

In [6]:
query = """
            SELECT DISTINCT id_aviao, modelo
            FROM aviao
            WHERE assentos > 200
            
        """

df = pd.read_sql_query(query, connection)
df

,id_aviao,modelo
0,4,Airbus A321
1,5,Airbus A321neo
2,6,Airbus A330-200
3,7,Airbus A330-900
4,15,Boeing 777-300ER
5,16,Boeing 787-9


## Consulta 3: Companhias aereas que não possuem nenhum avião com o selo B da anac


In [15]:
query = """
            SELECT DISTINCT id_companhia_aerea
            FROM companhia_aerea_aviao
            WHERE id_companhia_aerea NOT IN (
                SELECT id_companhia_aerea
                FROM companhia_aerea_aviao
                NATURAL JOIN aviao
                WHERE selo_anac = "B")
            
        """

df = pd.read_sql_query(query, connection)
df

,id_companhia_aerea
0,2


## Consulta 4: Orgão cuja soma das multas com bilhetes aéreos é maior

In [47]:
query = """
            SELECT DISTINCT codigo_orgao, nome_orgao, SUM(valor_multas) as soma_das_multas
            FROM orgao
            NATURAL JOIN orgao_bilhete
            NATURAL JOIN bilhete
            WHERE valor_multas > 0 
            GROUP BY codigo_orgao, nome_orgao
            ORDER BY soma_das_multas desc
            LIMIT 1
            
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao,soma_das_multas
0,70352,Secretaria-Geral de Administração,9562.89


## Consulta 5: Órgão que não comprou bilhetes aéreos

In [50]:
query = """
            SELECT codigo_orgao, nome_orgao
            FROM orgao
            WHERE codigo_orgao NOT IN (
                SELECT codigo_orgao
                FROM orgao
                NATURAL JOIN orgao_bilhete
            )
            
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao
0,26,Presidência da República
1,27,Gabinete de Segurança Institucional da Presidê...
2,46,Advocacia-Geral da União
3,2837,Casa Civil da Presidência da República
4,42673,Secretaria de Relações Institucionais
5,264359,Ministério das Comunicações
6,308798,Ministério das Cidades
7,308799,Ministério da Integração e do Desenvolvimento ...
8,308800,Ministério da Fazenda
9,308803,Ministério da Gestão e da Inovação em Serviços...
